In [1]:
import requests
import json
import pickle
import pandas as pd

In [2]:
#First request, API key removed for safety purposes
request=requests.get("https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants%20in%20Prague&key=MY-API-KEY")
request.status_code

200

In [31]:
#Converting the variable into a dictionary
data_dict=json.loads(request.text)

In [ ]:
#Inspecting the data
data_dict['results']
print(len(data_dict['results'])) #Only 20 restaurants => need to send additional requests for subsequent pages

In [43]:
#Preparing variables for additional requests
next_page_token=json.loads(request.text)['next_page_token']
restaurants=[]
restaurants.extend(json.loads(request.text)['results'])

In [ ]:
#Sending additional requests
i=0
while next_page_token:
    new_request=requests.get("https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants%20in%20Prague&key=MY-API-KEY&pageToken={next_page_token}")
    if new_request.status_code==200:
        restaurants.extend(json.loads(new_request.text)['results'])
        next_page_token=json.loads(new_request.text)['next_page_token']
        i += 1
    else:
        print(i)
        print(new_request.status_code)
        break
#Note: the loop was broken since I have reached the free quota
#Note 2: API key removed for safety purposes

#perhaps try timing the thing: There is a short delay between when a next_page_token is issued, and when it will become valid.

In [2]:
####### Honza's code ############
MY_API_KEY = "removed for safety"

#print(f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants%20in%20Prague&key={MY_API_KEY}")

dummy_request=requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants%20in%20Prague&key={MY_API_KEY}")
dummy_request.status_code

200

In [40]:
dummy_data_dict=json.loads(dummy_request.text)
#dummy_request.text
#dummy_data_dict

In [41]:
dummy_data_dict["results"]
dummy_data_dict["results"][0]["name"]

list_of_names = [(i['name']) for i in dummy_data_dict["results"]]
print(list_of_names)

['Restaurant Mlýnec', 'La Finestra in Cucina', 'Restaurant Pod Věží', 'Restaurace U Mlynáře', 'Isai Ramen', 'V Zátiší', 'Coda Restaurant', 'Restaurant Zvonice', 'Pastař', 'U modré kachničky II', 'Lokál', 'Czech Slovak Restaurant', 'Charles Bridge Restaurant', 'Old Prague Restaurant', 'Gran Fierro', 'St.Martin', 'Maitrea restaurace', 'Le Grill Restaurant', 'La Casa Argentina', 'Pivo & Basilico Caffe Restaurant']


In [42]:
#next page token
next_page_token1 = dummy_data_dict["next_page_token"]

dummy_request2=requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants%20in%20Prague&key={MY_API_KEY}&pagetoken={next_page_token1}")

In [43]:
#dummy_request2.text

dummy_data_dict2=json.loads(dummy_request2.text)
#dummy_data_dict2

dummy_data_dict2["results"]
dummy_data_dict2["results"][0]["name"]

list_of_names2 = [(i['name']) for i in dummy_data_dict2["results"]]
print(list_of_names)
print(list_of_names2)
#print(list_of_names3)
#print(list_of_names4)

#dummy_data_dict2["next_page_token"]

['Restaurant Mlýnec', 'La Finestra in Cucina', 'Restaurant Pod Věží', 'Restaurace U Mlynáře', 'Isai Ramen', 'V Zátiší', 'Coda Restaurant', 'Restaurant Zvonice', 'Pastař', 'U modré kachničky II', 'Lokál', 'Czech Slovak Restaurant', 'Charles Bridge Restaurant', 'Old Prague Restaurant', 'Gran Fierro', 'St.Martin', 'Maitrea restaurace', 'Le Grill Restaurant', 'La Casa Argentina', 'Pivo & Basilico Caffe Restaurant']
['Muc Dong - Vietnamese Restaurant Prague', 'Ichnusa Botega & Bistro', 'Hergetova Cihelna', 'BEEF BAR', 'GoodEats', 'Mangal Restaurant', "Black Bettie's Grill", 'Restaurace Mlejnice', 'Dinner in the Sky', 'Gruzínská restaurace, Gruzie', 'Restaurant U Balouna', 'Tři století', 'Augustine Restaurant & Garden', 'Spices Restaurant & Bar', 'Kogo Havelská ristorante', 'El Centro', 'Il Mulino restaurant', 'Restaurace Mincovna', 'U Modré Kachničky', 'Terasa U Zlaté studně']


In [45]:
next_page_token2 = dummy_data_dict2["next_page_token"]
dummy_request3=requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants%20in%20Prague&key={MY_API_KEY}&pagetoken={next_page_token2}")
dummy_data_dict3=json.loads(dummy_request3.text)
list_of_names3 = [(i['name']) for i in dummy_data_dict3["results"]]


In [46]:
print(list_of_names)
print(list_of_names2)
print(list_of_names3)

['Restaurant Mlýnec', 'La Finestra in Cucina', 'Restaurant Pod Věží', 'Restaurace U Mlynáře', 'Isai Ramen', 'V Zátiší', 'Coda Restaurant', 'Restaurant Zvonice', 'Pastař', 'U modré kachničky II', 'Lokál', 'Czech Slovak Restaurant', 'Charles Bridge Restaurant', 'Old Prague Restaurant', 'Gran Fierro', 'St.Martin', 'Maitrea restaurace', 'Le Grill Restaurant', 'La Casa Argentina', 'Pivo & Basilico Caffe Restaurant']
['Muc Dong - Vietnamese Restaurant Prague', 'Ichnusa Botega & Bistro', 'Hergetova Cihelna', 'BEEF BAR', 'GoodEats', 'Mangal Restaurant', "Black Bettie's Grill", 'Restaurace Mlejnice', 'Dinner in the Sky', 'Gruzínská restaurace, Gruzie', 'Restaurant U Balouna', 'Tři století', 'Augustine Restaurant & Garden', 'Spices Restaurant & Bar', 'Kogo Havelská ristorante', 'El Centro', 'Il Mulino restaurant', 'Restaurace Mincovna', 'U Modré Kachničky', 'Terasa U Zlaté studně']
['Zona Bistro', 'Staromáček', 'PAPRIKA - Mediterranean Kitchen & Bar', 'Aromi', 'Il Gusto', 'Pot-au-feu', 'El Emir 

In [47]:
#iterration 4
next_page_token3 = dummy_data_dict3["next_page_token"]
dummy_request4=requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?query=restaurants%20in%20Prague&key={MY_API_KEY}&pagetoken={next_page_token3}")
dummy_data_dict4=json.loads(dummy_request4.text)
list_of_names4 = [(i['name']) for i in dummy_data_dict3["results"]]


KeyError: 'next_page_token'

In [55]:
### FIND PLACE ###
#Restaurant Zvonice, 50.0851891, 14.4301079
place_request = requests.get(f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Restaurant%20Zvonice&inputtype=textquery&locationbias=circle%3A200%4050.0851891%2C14.4301079&fields=formatted_address%2Cname%2Crating%2Copening_hours&key={MY_API_KEY}")

place_request.status_code

200

In [56]:
place_request.content

#Yes, Ifound it 
#Note: I found restaurant returned by Google API, next step will be to find some restaurant scraped from zlatestranky (I am worried about possible different namings)

b'{\n   "candidates" : [\n      {\n         "formatted_address" : "Jind\xc5\x99i\xc5\xa1sk\xc3\xa1 33, 110 00 v\xc4\x9b\xc5\xbe-Nov\xc3\xa9 M\xc4\x9bsto, \xc4\x8cesko",\n         "name" : "Restaurant Zvonice",\n         "opening_hours" : {\n            "open_now" : true\n         },\n         "rating" : 4.5\n      }\n   ],\n   "status" : "OK"\n}\n'

In [10]:
#target: name : Repre restaurant, coordinates : {'latitude': 50.08553, 'longitude': 14.42718}, address : Nekázanka 4/857, 110 00 Praha 1-Nové Město okres Hlavní město Praha, Hlavní město Praha

place_request = requests.get(f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=Repre%20restaurant&inputtype=textquery&locationbias=circle%3A200%4050.08553%2C14.427189&fields=formatted_address%2Cname%2Crating%2Copening_hours&key={MY_API_KEY}")
place_request.status_code

200

In [11]:
place_request.content

#Ladies and gentleman, we got 'em


b'{\n   "candidates" : [\n      {\n         "formatted_address" : "Nek\xc3\xa1zanka 4, 110 00 Nov\xc3\xa9 M\xc4\x9bsto, \xc4\x8cesko",\n         "name" : "Repre restaurant",\n         "opening_hours" : {\n            "open_now" : false\n         },\n         "rating" : 4\n      }\n   ],\n   "status" : "OK"\n}\n'

In [75]:
#######################
#Filtering the data
#Empty list
restaurants_filtered=[]
#Keys
keys_new_dict=['name','address','opening_hours','place_id','price_level','rating','no_of_reviews','location']
keys_old_dict=['name','formatted_address','opening_hours','place_id','price_level','rating','user_ratings_total','geometry']
#Looping through restaurants to filter the data
for restaurant in restaurants:
    dict={}
    for key_new, key_old in zip(keys_new_dict, keys_old_dict):
        if key_new=='location': #Location is a dict within adict
            try:
                dict[key_new]=restaurant[key_old][key_new]
            except KeyError:
                dict[key_new]=None
        else: #Rest of the keys
            try:
                dict[key_new]=restaurant[key_old]
            except KeyError:
                dict[key_new]=None
    restaurants_filtered.append(dict)

In [78]:
#Saving the list for future use
with open('restaurants_google_maps_places_API.pkl', 'wb') as data_file:
    pickle.dump(restaurants_filtered, data_file)

In [7]:
#Code to read the file
with open('restaurants_google_maps_places_API.pkl', 'rb') as data_file:
    restaurants_filtered=pickle.load(data_file)

In [10]:
restaurants_filtered

[{'name': 'Restaurant Zvonice',
  'address': 'Jindřišská 33, 110 00 věž-Nové Město, Czechia',
  'place_id': 'ChIJmbWmIO2UC0cR_Gua9yvxWaw',
  'price_level': 3,
  'rating': 4.5,
  'no_of_reviews': 673,
  'location': (50.0851891, 14.4301079)},
 {'name': 'La Finestra in Cucina',
  'address': 'Platnéřská 90/13, 110 00 Staré Město, Czechia',
  'place_id': 'ChIJGxmor-iUC0cRdTmViA87nzs',
  'price_level': 3,
  'rating': 4.6,
  'no_of_reviews': 988,
  'location': (50.08735129999999, 14.4162553)},
 {'name': 'PAPRIKA - Mediterranean Kitchen & Bar',
  'address': 'Rumunská 8/16, 120 00 Vinohrady, Czechia',
  'place_id': 'ChIJvb0ESYmUC0cRmA9ecAWTok8',
  'price_level': 1,
  'rating': 4.7,
  'no_of_reviews': 1260,
  'location': (50.07383249999999, 14.4321147)},
 {'name': 'Bastion Prague Restaurant',
  'address': 'Horská 1751, 128 00 Nové Město, Czechia',
  'place_id': 'ChIJd7kyj2GUC0cRK8h-gH3M1aQ',
  'price_level': 3,
  'rating': 4.1,
  'no_of_reviews': 233,
  'location': (50.06786229999999, 14.4269231

In [9]:
#Adjusting the list before transforming it into a pandas DataFrame
#Putting the coordinates into a tuple
for restaurant in restaurants_filtered:
    restaurant['location']=tuple([restaurant['location']['lat'],restaurant['location']['lng']])
#Removing opening hours since they do not seem to be informative and we need to decrease the file size
for restaurant in restaurants_filtered:
    restaurant.pop('opening_hours')

In [11]:
#Creating a pandas DataFrame
df=pd.DataFrame(restaurants_filtered)

In [12]:
#Writing into a csv file
df.to_csv('restaurants_google_maps_places_API.csv')

In [14]:
#Trying to read the file
df2=pd.read_csv('restaurants_google_maps_places_API.csv')